<img src="../img/crowdlearning-etic.png" alt="Logo ETIC" align="right">


<h1><font color="#004D7F" size=6>Transformaciones</font></h1>

<br>
<br>
<br>
<br>
<div style="text-align: right">
<font color="#004D7F" size=3>Antonio Jesús Gil</font><br>
<font color="#004D7F" size=3>Fundamentos de Machine Learning</font><br>

</div>

# <font color="#004D7F">Transformaciones con SciKit Learn</font>

Los transformadores son aquellos algoritmos y funciones que toman como datos de entrada las variables de nuestro problema y devuelven otras variables.
Estas transformaciones suelen aplicarse antes de un algoritmo de clasificación/regresión, y todas aquellas operaciones que se ejecutan antes de un algoritmo de predicción reciben el nombre de preprocesamientos.

En este apartado vamos a ver cómo funcionan las trasnformaciones en SciKit Learn. Para ello, vamos a aplicar distintas transformaciones a las bases de datos vistas. En SciKit Learn, todos los transformadores implementan dos funciones principales, `fit` y `transform`, y la que combina a ambos, `fit_transform`. 

## <font color="#004D7F">Impute</font>

Una de las trasnformaciones más __básicas__ consiste en la sustitución de valores perdidos o `NaN's` de la base de datos, ya que muchos algoritmos no son capaces de manejarlos. Una de las formas de realizar esta sustitución de valores perdidos consiste en utilizar la media (para valores continuos) o la moda (caso discreto) con los casos conocidos y aplicar dicha métrica a los `NaN's` de la variable.

El módulo [__Impute__ en SciKit Learn](https://scikit-learn.org/stable/modules/impute.html) provee dos métodos para calcular y transformar:

> La clase `sklearn.impute.SimpleImputer` se encarga de calcular y modificar los datos de entrada en una única dimensión de una variable.

> `sklearn.impute.sklearn.IterativeImputer` utiliza el conjuto completo de características para estimar valores perdidos.


Veamos cómo se utiliza en el caso de Wisconsin.

### <font color="#004D7F">Valores perdidos en Wisconsin</font>

In [ ]:
import pandas as pd
wisconsin = pd.read_csv('wisconsin.csv')
wisconsin.head(5)

In [ ]:
# Lo primero es separar los atributos de la clase.
wisconsin_data = wisconsin.drop('label',1)
wisconsin_target = wisconsin['label']

Comprobamos si existen valores perdidos `NaN's` Indicamos 3 métodos para mayor aprendizaje.

In [ ]:
wisconsin_data.isnull().values.any()

In [ ]:
wisconsin_data.isnull().sum()

In [ ]:
import numpy as np
# Después comprobamos los NaN's de nuestros datos, esto podemos hacerlo con Numpy.
print(np.sum(np.isnan(wisconsin_data)))

Solo la variable `bareNuclei` tiene valores perdidos, por lo que vamos a aplicar el `SimpleImputer` y ver que resultados se obtienen. Recordar que contiene tanto las funciones `fit` como `transform` y `fit_transform`

In [ ]:
from sklearn.impute import SimpleImputer

# Creamos un objeto de la clase Imputer con los parámetros por defecto
imp = SimpleImputer()

# Llamamos a fit, para que aprenda la media con la que tiene que rellenar los NaN's y le pasamos los datos
imp.fit(wisconsin_data)

# Finalmente, transformamos los datos con las medias aprendidas.
wisconsin_trans = imp.transform(wisconsin_data)

# Aunque le hayamos pasado un pandas a SciKit Learn, este nos devuelve un numpy array, por lo que volvemos a 
# trasnformarlo a pandas
wisconsin_trans = pd.DataFrame(wisconsin_trans, columns = wisconsin_data.columns)

# Si volvemos a comprobar los NaN's en la nueva base de datos, vemos que ya no hay
print(np.sum(np.isnan(wisconsin_trans)))

Tanto el método anterior como este son equivalentes

In [ ]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(wisconsin_data)
wisconsin_trans_1 = imp.transform(wisconsin_data)
wisconsin_trans_1 = pd.DataFrame(wisconsin_trans, columns = wisconsin_data.columns)

# Si volvemos a comprobar los NaN's en la nueva base de datos, vemos que ya no hay
print(np.sum(np.isnan(wisconsin_trans_1)))

Es posible realizar el proceso en un sólo paso gracias a la función fit_transform y el resultado es el mismo

In [ ]:
imp = SimpleImputer()
wisconsin_fit_trans = imp.fit_transform(wisconsin_data)
wisconsin_fit_trans = pd.DataFrame(wisconsin_fit_trans, columns = wisconsin_data.columns)

# np.all nos devuelve true sólo si todas las comparaciones son ciertas, esto querrá decir que tanto el 
# fit y transform como el fit_transform han hecho lo mismo
print(np.all(wisconsin_fit_trans == wisconsin_trans))

En este caso, los valores perdidos han sido sustituidos por la media de los valores conocidos en `bareNuclei`. Si vamos a la documentación del `Imputer` en https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html, podremos observar los diferentes parámetros de configuración del algoritmo. Si queremos modificar el comportamiento del algoritmo deberemos definir estos parámetros
en el constructor cuando creemos el objeto. 

#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicio 1: Valores perdidos por la moda en Wisconsin</font>

En este primer ejercicio se pide utilizar la mediana en vez de la media para rellenar los valores perdidos de `wisconsin_data`.

In [ ]:
# ToDo...

### <font color="#004D7F">Normalización por la media en MNIST</font>

En teoría se ha visto cómo la normalización por la media ayuda a los algoritmos lineales a encontrar antes una solución. Este algoritmo también se utiliza en el procesamiento de imágenes y viene implementado en SciKit Learn con la clase `sklearn.preprocessing.StandardScaler`. Este transformador en su configuración por defecto, aplica la __normalización por la media__ de los atributos. 

Esto significa que pondera todas las variables por igual, esto significa que si disponemos de variables con 2 digitos, por ejemplo, edad y otra con 5 digitos, por ejemplo salario, se corre el peligro de que el modelo le otorgue mucha más importancia al salario que a la edad, y eso no es necesariamente exacto.

#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicio 2: Dígitos de MNIST normalizados por la media</font> 

Se pide de la misma forma que en el caso anterior, se pruebe este transformador y que se obtenga una nueva base de datos mnist_scaler y que se comparen los dígitos imprimidos antes con los transformados.

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, data_home=".")

# la base de datos descargada consiste en un diccionario con diferentes parámetros, de los
# cuales sólo nos quedamos con data y target.

# data consiste en una matriz donde cada fila es una imagen y las columnas representan los
# pixels que la componen
mnist_data = mnist['data']
print("Número de imágenes: {0}\tPixels por imagen: {1}".format(mnist_data.shape[0],mnist_data.shape[1]))

# target contiene las etiquetas de cada una de las imágenes, donde el valor de la clase 
# corresponde con el dígito que se muestra en la imagen
mnist_target = mnist['target']
print("Número de etiquetas: {0}".format(mnist_target.shape[0]))

In [ ]:
# crear un objeto StandardScaler y aplicar la función fit y transform a mnist_data
from sklearn.preprocessing import StandardScaler
# ToDo...

Ejemplo de MNIST classfification using multinomial logistic + L1

https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html#sphx-glr-auto-examples-linear-model-plot-sparse-logistic-regression-mnist-py

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

print(__doc__)

# Author: Arthur Mensch <arthur.mensch@m4x.org>
# License: BSD 3 clause

# Turn down for faster convergence
t0 = time.time()
train_samples = 5000

# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)

random_state = check_random_state(0)
permutation = random_state.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]
X = X.reshape((X.shape[0], -1))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size=10000)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Turn up tolerance for faster convergence
clf = LogisticRegression(C=50. / train_samples,
                         multi_class='multinomial',
                         penalty='l1', solver='saga', tol=0.1)
clf.fit(X_train, y_train)
sparsity = np.mean(clf.coef_ == 0) * 100
score = clf.score(X_test, y_test)
# print('Best C % .4f' % clf.C_)
print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

coef = clf.coef_.copy()
plt.figure(figsize=(10, 5))
scale = np.abs(coef).max()
for i in range(10):
    l1_plot = plt.subplot(2, 5, i + 1)
    l1_plot.imshow(coef[i].reshape(28, 28), interpolation='nearest',
                   cmap=plt.cm.RdBu, vmin=-scale, vmax=scale)
    l1_plot.set_xticks(())
    l1_plot.set_yticks(())
    l1_plot.set_xlabel('Class %i' % i)
plt.suptitle('Classification vector for...')

run_time = time.time() - t0
print('Example run in %.3f s' % run_time)
plt.show()

## <font color="#004D7F">Generación de nuevas variables</font>

En este apartado vamos a hacer ciertas modificaciones usando la función `sklearn.preprocessing.PolynomialFeatures`. Este __transformador__ aumenta el número de variables aplicando funciones polinomiales en base al grado (parámetro de entrada) de la siguiente forma: 
 * Si tenemos dos atributos [a, b], las nuevos atributos polinomiales de grado 2 serían [1, a, b, a^2, ab, b^2].
 
#### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicio 3: Nuevas variables polinomiales de grado 2 en Pima</font> 

La forma de utilizar el algoritmo es igual que con `Imputer`, por eso se pide que utilizando `fit` y `transform` o `fit_transform`, se obtenga de `pima_data` una nueva base de datos llamada `pima_pol` aplicando `PolynomialFeatures`. Imprimir cuantas variables tiene ahora la base de datos y su nombre. Para obtener el nombre de las nuevas características mirar la función `get_feature_names` de la documentación en http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

Se pide: 

Aplicar este transformador en una nueva variable denominada `pima_pol` de igual manera que en el ejercio anterior.
Se obtendran muchas variables nuevas a partir de las originales ¿nombre? ¿como se han generado? ¿como estan ordenadas?

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
# ToDo